In [1]:
IN1 = !cat aoc/2023/day1

In [2]:
digits = lambda s: [(i, c) for i, c in enumerate(s) if str.isdigit(c)]
first_and_last = lambda a: (a[0][1], a[-1][1])
get_num = lambda ds: int("".join(first_and_last(ds)))

ans1 = sum(get_num(digits(s)) for s in IN1)

########################################

ws = {"one": "1", "two": "2", "three": "3", "four": "4", "five": "5", "six": "6", "seven": "7", "eight": "8", "nine" : "9"}

def unword(s, items): 
    for it in items:
        s = s.replace(*it)
    return digits(s)

# No digit word can overlap with itself, so the first and last word
# will be unharmed in at least one of the replacement directions
digits_both_ways = lambda s: sorted(unword(s, ws.items()) + unword(s, reversed(ws.items())))

ans2 = sum(get_num(digits_both_ways(s)) for s in IN1)

ans1, ans2

(55090, 54845)

In [3]:
IN2 = !cat aoc/2023/day2
IN2 = [r.split(": ")[1] for r in IN2]

In [5]:
from functools import reduce
from operator import mul, le

BAG = {"red": 12, "green": 13, "blue": 14}

games = lambda r: [dict(reversed(x.split(" ")) for x in ii.strip().split(", ")) for ii in r.split(";")]
vals = lambda g: [int(g.get(col, 0)) for col in BAG]
maxv = lambda gs: map(max, *map(vals, gs))

ans1 = sum((i+1) for i, r in enumerate(IN2) if all(map(le, maxv(games(r)), vals(BAG))))

########################################

power = lambda v: reduce(mul, v, 1)

ans2 = sum(power(maxv(games(r))) for r in IN2)

ans1, ans2

(2716, 72227)

In [6]:
IN3 = !cat aoc/2023/day3

In [7]:
import re

M, N = len(IN3), len(IN3[0])
board = [(i, j) for i in range(M) for j in range(N)]
x3x3 = lambda p: [(p[0]+x, p[1]+y) for x in [-1, 0, 1] for y in [-1, 0, 1]]
val = lambda p: IN3[p[0]][p[1]]

valid = lambda cs: (0, 0) <= cs <= (M-1, N-1)

neighb = lambda p: {q for q in x3x3(p) if valid(q)}
fig_neighb = lambda fig: set.union(*[neighb(p) for p in fig])

all_sym = {p for p in board if val(p) not in "0123456789."}

row_nums = lambda r: [(range(*m.span()), m.group()) for m in re.finditer("([0-9]+)", r)]
row_neighbs = lambda i, r: [(num, fig_neighb((i, j) for j in span)) for span, num in row_nums(r)]
all_nums = sum([row_neighbs(i, r) for i, r in enumerate(IN3)], [])

ans1 = sum(int(n) for n, nb in all_nums if nb & all_sym)

########################################

all_gears = {p for p in board if val(p) == "*"}
gear_nums = [[int(n) for n, nb in all_nums if p in nb] for p in all_gears]

ans2 = sum(v[0]*v[1] for v in gear_nums if len(v) == 2)
                
ans1, ans2            

(520019, 75519888)

In [8]:
IN4 = !cat aoc/2023/day4

In [17]:
cards = [[set(s.split()) for s in r.split("|")] for r in IN4]
counts = [len(w & n) for w, n in cards]

ans1 = sum(int(2**(c-1)) for c in counts)

########################################

from functools import reduce

ncopy = lambda its: 1 + sum(it.pop() for it in its if it)
win = lambda t, c: [t[0] + (won:=ncopy(t[1])), t[1] + [[won]*c]]

ans2 = reduce(win, counts, (0, []))[0]

#### Readable immutable version of part 2

total, clone_lists = 0, []
for count in counts:
    copies_of_n = 1 + sum(cl[0] for cl in clone_lists if cl)
    total += copies_of_n
    
    clone_lists = [cl[1:] for cl in clone_lists if cl[1:]] + [[copies_of_n] * count]
    
ans2_1 = total

ans1, ans2, ans2_1

(21821, 5539496, 5539496)

In [11]:
IN5 = !cat aoc/2023/day5

In [22]:
from itertools import groupby
from functools import reduce

seeds = [int(x) for x in IN5[0].split()[1:]]

parse_map = lambda g: [[int(x) for x in s.split()] for s in list(g)[1:]]
maps = [parse_map(g) for k, g in groupby(IN5[2:], key=bool) if k]

in_range = lambda r, x: r[0] <= x < r[0]+r[-1]
fwd_map  = lambda p, m: next((p-r[1]+r[0] for r in m if in_range(r[1:], p)), p)
fwd_list = lambda ps, m: [fwd_map(p, m) for p in ps]
end_pos  = lambda ps: reduce(fwd_list, maps, ps)

ans1 = min(end_pos(seeds))

########################################

back_map = lambda p, m: next((p-r[0]+r[1] for r in m if in_range(r, p)), p)
back_list = lambda ps, m: [back_map(p, m) for p in ps]

# There's probably some better dynamic programming algo.
# The main insight is that for a given starting range,
# the minimum will be achieved either at its starting point,
# or at one of the starting points of the intermediate map ranges.
# To prove, assume the absolute min happens at K, which does not
# map to any starting point at any stage - but then K-1 is
# guaranteed to be in the same range as K on every map, including the starting one
# and ends up one lower at the end, a contradiction.
# So we pull all starting points back to the seed level,
# and then check en masse for their ending positions.

seed_ranges = list(zip(seeds[::2], seeds[1::2]))

map_starts = lambda m: [r[1] for r in m]
pull = lambda prev_starts, m: back_list(prev_starts, m) + map_starts(m)
pre_starts = reduce(pull, reversed(maps), [])

ends = dict(zip(pre_starts, end_pos(pre_starts)))

ends_for_range = lambda r: [ends[s] for s in pre_starts if in_range(r, s)]
min_for_range = lambda r: min(end_pos(r) + ends_for_range(r))
ans2 = min(min_for_range(r) for r in seed_ranges)

ans1, ans2

(51752125, 12634632)

In [97]:
IN6 = !cat aoc/2023/day6

In [98]:
races = [[int(x) for x in r.split()[1:]] for r in IN6]
root = lambda t,d: t/2 + (t*t/4 - d) ** 0.5 - 0.001
wins = lambda p, r: p * (int(root(*r)) - int(r[0]-root(*r)))

ans1 = reduce(wins, zip(*races), 1)

########################################

race = [int("".join(r.split()[1:])) for r in IN6]
ans2 = wins(1, race)

ans1, ans2

(252000, 36992486)

In [152]:
IN7 = !cat aoc/2023/day7
BIN7 = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".split("\n")

In [159]:
from collections import Counter
order = "23456789TJQKA"

htype = lambda s: sorted(Counter(s).values(), reverse=True)
hand = lambda s: [order.find(c) for c in s]

hands = lambda typer: [(typer(s), hand(s), int(bid)) for s, bid in map(str.split, IN7)]
winnings = lambda typer: sum((i+1)*h[2] for i, h in enumerate(sorted(hands(typer))))

ans1 = winnings(htype)

########################################

order = "J23456789TQKA"

noj = lambda s: htype(s.replace("J", "")) or [0]
jtype = lambda s: [s.count("J") + noj(s)[0]] + noj(s)[1:]

ans2 = winnings(jtype)

ans1, ans2

(248453531, 248781813)